Estimated Runtime: 15-20 Hours

Required Packages: numpy, pandas, matplotlib, sklearn

License: MIT

Author: Tommy Liu
    

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import matplotlib as mpl
from sklearn.linear_model import Ridge
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import ResidualDecomposition as RD

np.random.seed(0)

In [2]:
data = pd.read_csv("Datasets/P_BostonHousing.csv") 
X = data.iloc[:,:-1].values
Y = data.iloc[:,-1].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.7)

In [3]:
lr = RandomForestRegressor(n_estimators=1000)
lr.fit(X, Y)

RandomForestRegressor(n_estimators=1000)

In [4]:


# t1 = time.time()

# rshap_model = RD.ResidualDecompositionAsymmetric()
# rshap_model.fit(X_train, X_test, Y_train, Y_test, lr, 1000)
# rcomposition = rshap_model.get_composition()
# rcontribution = rshap_model.get_contribution()

# t2 = time.time()

In [5]:
# dshap = RD.Data_Shapley()
# dshap.fit(X_train, X_test, Y_train, Y_test, mean_squared_error, lr, 1000)

In [6]:
# dshapvals = dshap.shap_vals
# rshapvals = rshap

In [7]:
dshapvals = np.genfromtxt("Data/RSHAP_Boston_RF1000.csv", delimiter=',')

In [8]:
lr.fit(X_train, Y_train)
sc = mean_squared_error(lr.predict(X_test), Y_test)

print(sc, np.sum(dshapvals, axis=0))

9.468680044342149 [ 2.04102512  0.93370552 -0.14018951  0.69461224 -1.1625027  -2.40064348
 -2.09641008 -4.43862597  0.59359207  0.39229115  2.61746036  1.52130295
 -0.54767568  0.06560862  1.87713231  0.5223736  -0.84868468  1.15625847
 -0.39936599  1.64824085  0.90912452  0.18625611  0.56478099  0.53008745
  0.57924957  1.24764254  1.02959794  0.36625638  1.77041776  0.9797643
  1.56896648  4.56664253  1.65618904  1.57177883  0.8034974   1.8988484
  0.67449083 -0.05243565 -3.65265596 -1.46801772 -0.60222184  1.92132938
 -1.27527197 -0.72595512  0.2699277   1.05558084  0.43804442  1.77474466
  2.30200696  0.15998658  0.91787855  1.30913314 -0.36958736 -2.26609668
  0.5042624  -0.8032944  -1.16210997 -0.32071629  0.09961767  1.12168543
  1.14827839  2.56790429  1.42168046  0.44226526 -0.03379005  0.46401842
  0.90999781 -1.19073291  2.33700824 -0.07852962 -0.39676563 -0.94526395
  0.15299006  0.30403065 -0.14869221  0.97902499  0.93000849  0.46398509
 -0.19824035  0.56199121 -0.7058315

In [9]:
np.sum(dshapvals)

4.561106418868405

In [10]:
rpred = np.sum(rshap, axis=0)

r_contribution = (((rshap ** 2) * -np.sign(rpred))) 

NameError: name 'rshap' is not defined

In [ ]:
r_contribution.shape, rshap.shape

In [ ]:
plt.xlabel("RSHAP Values")
plt.ylabel("DSHAP Values")

plt.scatter(np.mean(r_contribution, axis=1), dshapvals, c=Y_train)

In [ ]:
# Removing the 'most' important instances

dshapscore = []
rshapscore = []
randomscore = []

dsorted = np.argsort(dshapvals)[::-1]
rsorted = np.argsort((np.mean(rshapvals, axis=1)))
ranted = np.random.permutation(np.arange(0, X_train.shape[0]))

for i in reversed(range(X_train.shape[0]//2, X_train.shape[0])):
    Xs = X_train[dsorted[:i]]
    Ys = Y_train[dsorted[:i]]
    
    lr.fit(Xs, Ys)
    dshapscore.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[rsorted[:i]]
    Ys = Y_train[rsorted[:i]]
    
    lr.fit(Xs, Ys)
    rshapscore.append(mean_squared_error(lr.predict(X_test), Y_test))
    


for j in range(0, 5):
    tempscore = []
    for i in reversed(range(X_train.shape[0]//2, X_train.shape[0])):
        Xs = X_train[ranted[:i]]
        Ys = Y_train[ranted[:i]]

        lr.fit(Xs, Ys)
        tempscore.append(mean_squared_error(lr.predict(X_test), Y_test))
    randomscore.append(tempscore)
    print(j)

In [ ]:
plt.ylabel("MSE Score")

plt.title("Removing Largest Contributors to Loss")

plt.plot(dshapscore, label='dshap')
plt.plot(rshapscore, label='rshap')
plt.plot(np.mean(randomscore, axis=0), label='random')

plt.legend()

In [ ]:
dshapscore = []
rshapscore = []
randomscore = []

dsorted = np.argsort(np.abs(dshapvals))[::-1]
rsorted = np.argsort(np.abs(np.mean(r_contribution, axis=1)))[::-1]
ranted = np.random.permutation(np.arange(0, X_train.shape[0]))

for i in reversed(range(X_train.shape[0]//2, X_train.shape[0])):
    Xs = X_train[dsorted[:i]]
    Ys = Y_train[dsorted[:i]]
    
    lr.fit(Xs, Ys)
    dshapscore.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[rsorted[:i]]
    Ys = Y_train[rsorted[:i]]
    
    lr.fit(Xs, Ys)
    rshapscore.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[ranted[:i]]
    Ys = Y_train[ranted[:i]]
    
    lr.fit(Xs, Ys)
    randomscore.append(mean_squared_error(lr.predict(X_test), Y_test))

In [ ]:
plt.ylabel("MSE Score")

plt.title("Removing Smallest Contributors to Loss")
plt.plot(dshapscore, label='dshap')
plt.plot(rshapscore, label='rshap')
plt.plot(randomscore, label='random')

plt.legend()

In [ ]:
# Removing the 'most' important instances

dshapscore1 = []
dshapscore2 = []
rshapscore1 = []
rshapscore2 = []
randomscore = []

dsorted = np.argsort(np.abs(dshapvals))
dsortedr = np.argsort(np.abs(dshapvals))[::-1]
rsorted = np.argsort(np.mean(r_contribution, axis=1))
rsortedr = np.argsort(np.mean(r_contribution, axis=1))[::-1]

for i in reversed(range(X_train.shape[0]//2, X_train.shape[0])):
    Xs = X_train[dsorted[:i]]
    Ys = Y_train[dsorted[:i]]
    
    lr.fit(Xs, Ys)
    dshapscore1.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[dsortedr[:i]]
    Ys = Y_train[dsortedr[:i]]
    
    lr.fit(Xs, Ys)
    dshapscore2.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[rsorted[:i]]
    Ys = Y_train[rsorted[:i]]
    
    lr.fit(Xs, Ys)
    rshapscore1.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[rsortedr[:i]]
    Ys = Y_train[rsortedr[:i]]
    
    lr.fit(Xs, Ys)
    rshapscore2.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[ranted[:i]]
    Ys = Y_train[ranted[:i]]
    
    lr.fit(Xs, Ys)
    randomscore.append(mean_squared_error(lr.predict(X_test), Y_test))


In [ ]:
randomscore = []
for j in range(0, 10):
    ranted = np.random.permutation(np.arange(0, X_train.shape[0]))
    
    tempscore = []
    for i in reversed(range(X_train.shape[0]//2, X_train.shape[0])):
        Xs = X_train[ranted[:i]]
        Ys = Y_train[ranted[:i]]

        lr.fit(Xs, Ys)
        tempscore.append(mean_squared_error(lr.predict(X_test), Y_test))
    randomscore.append(tempscore)
    print(j)

In [ ]:
plt.ylabel("MSE Score")

plt.plot(rshapscore1, label='Largest Decreasers of Loss (RSHAP)', c="#fde725")
plt.plot(rshapscore2, label='Largest Increasers of Loss (RSHAP)', c="#5ec962")
plt.plot(dshapscore1, label='Largest Decreasers of Loss (DSHAP)', c="#21918c")
plt.plot(dshapscore2, label='Largest Increasers of Loss (DSHAP)', c="#3b528b")
plt.plot(np.mean(randomscore, axis=0), label='Random', c="#440154")

plt.legend()

In [ ]:
# Removing the 'most' important instances

dshapscore1 = []
dshapscore2 = []
rshapscore1 = []
rshapscore2 = []

dsorted = np.argsort((dshapvals))
dsortedr = np.argsort((dshapvals))[::-1]
rsorted = np.argsort(np.mean(r_contribution ** 2, axis=1))
rsortedr = np.argsort(np.mean(r_contribution ** 2, axis=1))[::-1]

for i in reversed(range(X_train.shape[0]//2, X_train.shape[0])):
    Xs = X_train[dsorted[:i]]
    Ys = Y_train[dsorted[:i]]
    
    lr.fit(Xs, Ys)
    dshapscore1.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[dsortedr[:i]]
    Ys = Y_train[dsortedr[:i]]
    
    lr.fit(Xs, Ys)
    dshapscore2.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[rsorted[:i]]
    Ys = Y_train[rsorted[:i]]
    
    lr.fit(Xs, Ys)
    rshapscore1.append(mean_squared_error(lr.predict(X_test), Y_test))
    
    Xs = X_train[rsortedr[:i]]
    Ys = Y_train[rsortedr[:i]]
    
    lr.fit(Xs, Ys)
    rshapscore2.append(mean_squared_error(lr.predict(X_test), Y_test))



In [ ]:
plt.figure(figsize=(14,10))

plt.plot(rshapscore1, label='Largest Contribution', c="#fde725", linewidth=3)
plt.plot(rshapscore2, label='Smallest Contribution', c="#5ec962", linewidth=3)
plt.plot(dshapscore1, label='Largest Value', c="#21918c", linewidth=3)
plt.plot(dshapscore2, label='Smallest Value', c="#3b528b", linewidth=3)
plt.plot(np.mean(randomscore, axis=0), label='Random', c="#440154", linewidth=3)

plt.legend(fontsize=16)
plt.xticks([0,X_train.shape[0]//2 - X_train.shape[0]//4, X_train.shape[0]//2], [100, 75, 50], fontsize=20)
plt.xlabel("Percentage of Remaining Training Data", fontsize=20)
plt.ylabel("MSE Score", fontsize=20)
plt.yticks(fontsize=16)

plt.savefig("Figures/boston_dshap_vs_rshap_rf.pdf", bbox_inches='tight')

plt.xlim([0, X_train.shape[0]//2])

In [ ]:
plt.plot(np.sort(dshapvals))

In [ ]:
np.sign(rpred).shape

In [ ]:
r_contribution.shape, rshap.shape

In [ ]:
rpred = np.sum(rshap, axis=0)

r_contribution = ((rshap * -np.sign(rpred)))

In [ ]:
r_contribution.shape

In [ ]:
sY = np.argsort(Y_train)

plt.plot(Y_train[sY], np.mean(r_contribution, axis=1)[sY])

In [ ]:
AA = (r_contribution @ r_contribution.T)

In [ ]:
itotal_contributions = np.sum(np.abs(r_contribution), axis=1)

In [ ]:
plt.scatter(Y_train, itotal_contributions, c=Y_train)

plt.xlabel("Energy of Fermi Level")
plt.ylabel("Total Contribution of instance")

In [ ]:
itotal_composition = np.sum(np.abs(rshap), axis=0)

In [ ]:
plt.figure(figsize=(14,10))

plt.scatter(Y_test, itotal_composition, c=Y_test, s=40)

plt.xlabel("Energy of Fermi Level")
plt.ylabel("Total Composition of instance")

In [ ]:
plt.scatter(np.mean(rshap, axis=0), np.mean(r_contribution, axis=1), c=Y_test)

In [ ]:
lr = RandomForestRegressor(n_estimators=200, max_depth=7)
lr.fit(X, Y)

t1 = time.time()

rshap_model = class_files.residual_shap_MC(X, Y, lr, niter=1000)
rshap = rshap_model.phi

t2 = time.time()

In [ ]:
(t2-t1 ) / (60 * 60)

In [ ]:
rshap 36 * 42

In [ ]:
rpred = np.sum(rshap, axis=0)

rshap_norm = ((rshap.T * -np.sign(rpred))).T 

r_contribution = ((rshap.T * -np.sign(rpred))).T 

In [ ]:
plt.figure(figsize=(14,10))

plt.scatter(np.mean(rshap, axis=0), np.mean(r_contribution, axis=1), c=Y, s=40)

plt.grid(True)

plt.xlabel("Composition Mean", fontsize=20)
plt.ylabel("Contribution Mean", fontsize=20)
plt.axvline(0, c='red')
plt.axhline(0, c='orange')

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=16)

plt.savefig("Figures/boston_rshap_rf_1000.pdf", bbox_inches='tight')

In [ ]:
iso_forest = IsolationForest(n_estimators=1000)
iso_forest.fit(r_contribution)

iso_predicted = iso_forest.predict(r_contribution)

iso_int = np.where(iso_predicted == -1)
iso_out = np.where(iso_predicted == 1)

plt.figure(figsize=(14,10))

plt.xlabel("Composition Mean", fontsize=20)
plt.ylabel("Contribution Mean", fontsize=20)
plt.axvline(0, c='red')
plt.axhline(0, c='orange')

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.scatter(np.mean(rshap, axis=0)[iso_int], np.mean(r_contribution, axis=1)[iso_int], c='r', s=40, label='Outliers')
plt.scatter(np.mean(rshap, axis=0)[iso_out], np.mean(r_contribution, axis=1)[iso_out], c='orange', s=40, label='Inliers')

plt.grid(True)



In [ ]:
iso_forest = IsolationForest(n_estimators=1000)
iso_forest.fit(X)

iso_predicted = iso_forest.predict(X)

iso_int = np.where(iso_predicted == -1)
iso_out = np.where(iso_predicted == 1)

plt.figure(figsize=(14,10))

plt.xlabel("Composition Mean", fontsize=20)
plt.ylabel("Contribution Mean", fontsize=20)
plt.axvline(0, c='red')
plt.axhline(0, c='orange')

plt.xticks(fontsize=16)
plt.yticks(fontsize=16)

plt.scatter(np.mean(rshap, axis=0)[iso_int], np.mean(r_contribution, axis=1)[iso_int], c='r', s=40, label='Outliers')
plt.scatter(np.mean(rshap, axis=0)[iso_out], np.mean(r_contribution, axis=1)[iso_out], c='orange', s=40, label='Inliers')
plt.legend(fontsize=16)

plt.grid(True)

In [ ]:
lr = RandomForestRegressor(n_estimators=200, max_depth=7)
lr.fit(X, Y)

res = lr.predict(X) - Y

plt.scatter(np.sum(rshap, axis=0), res, c=Y)

In [ ]:
Y[180]

In [ ]:
cmap = mpl.cm.get_cmap("viridis")

plt.figure(figsize=(20, 8))

i_instance = 180

i_residual = res[i_instance]

composition_values = rshap[:,i_instance]
composition_orders = np.argsort(composition_values)
composition_ordered = composition_values[composition_orders]
Y_ordered = Y[composition_orders]

less_zero = np.where(composition_ordered < 0)[0]
more_zero = np.where(composition_ordered > 0)[0]  
current_Y = np.sum(composition_ordered[less_zero])

for i in range(0, more_zero.shape[0]):
    current_index = more_zero[i]
    current_height = np.abs(composition_ordered[current_index])
    Y_color = cmap(Y_ordered[current_index] / 50)
    plt.barh(0, current_height, 0.1, current_Y, color=Y_color)
    current_Y += current_height
    
plt.axvline(current_Y, c='r', linewidth=5)

for i in range(0, less_zero.shape[0]):
    current_index = less_zero[i]
    current_height = np.abs(composition_ordered[current_index])
    Y_color = cmap(Y_ordered[current_index] / 50)
    plt.barh(0, current_height, 0.1, current_Y, color=Y_color)
    current_Y += current_height
    
plt.xlabel("Residual Value", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks([])

from matplotlib.cm import ScalarMappable
sm = ScalarMappable(cmap=cmap, norm=plt.Normalize(0,50))
sm.set_array([])

plt.barh(0,0, 0.2)

cbar = plt.colorbar(sm)
cbar.set_label('Y-Value of Contributing Instance', rotation=270,labelpad=25, fontsize=20)
plt.xlim(-25, 25)